# -------------------------------------------------------------
# AG2 Multi-Agent System 
# -------------------------------------------------------------

# Install AG2 

In [1]:
!pip install "ag2[openai]" -q


# -------------------------------------------------------------
# LLM Configuration for AG2 Multi-Agent System
# -------------------------------------------------------------

In [2]:
import os
from autogen import LLMConfig

llm_config = LLMConfig(config_list={
    "api_type": "openai",      
    "model": "gpt-4o",         
    "api_key": os.environ["OPENAI_API_KEY"],  
    "temperature": 0.2          
})

print(" LLM configuration loaded successfully.")



 LLM configuration loaded successfully.


# -------------------------------------------------------------
# Context Variables and Agent Skeletons
# -------------------------------------------------------------

In [3]:
from autogen.agentchat.group import ContextVariables
from autogen import ConversableAgent

context_variables = ContextVariables(data={
    "user_query": None,          
    "explained_terms": None,      
    "mp_query": None,            
    "mp_results": None,           
    "final_conclusion": None      
})


agent_explainer = ConversableAgent(
    name="AgentA_Explainer",
    system_message=(
        "You are Agent A (Explainer). "
        "Your job is to interpret the user's material design query, "
        "explain technical terms such as bandgap or crystal structure, "
        "and save your summary in context_variables['explained_terms']. "
        "Do not perform external searches."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)

agent_retriever = ConversableAgent(
    name="AgentB_MaterialsRetriever",
    system_message=(
        "You are Agent B (Retriever). "
        "Using both the user's query and Agent A's explanation, "
        "formulate a structured query for the Materials Project API, "
        "store it in context_variables['mp_query'], "
        "retrieve the materials, and store the raw results in "
        "context_variables['mp_results']."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)

agent_analyzer = ConversableAgent(
    name="AgentC_Analyzer",
    system_message=(
        "You are Agent C (Analyzer). "
        "Analyze the retrieved materials in context_variables['mp_results'], "
        "extract insights about which materials meet the criteria (e.g., bandgap > 3), "
        "and write your summary in context_variables['final_conclusion']."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)


# -------------------------------------------------------------
# Materials Project Tool integration 
# -------------------------------------------------------------

In [4]:
from autogen.tools import tool
from pymatgen.ext.matproj import MPRester
import os

@tool(name="materials_project_query", description="Retrieve materials from the Materials Project database.")
def materials_project_query(criteria: dict, properties: list):
    """
    Queries the Materials Project database for materials that match the given criteria.
    """
    api_key = os.getenv("MP_API_KEY")
    if not api_key:
        raise ValueError("Materials Project API key not found. Please set MP_API_KEY in your environment.")
    
    with MPRester(api_key) as mpr:
        results = mpr.query(criteria=criteria, properties=properties)
    return results


# Recreate Agent B (Retriever) with the updated system message referencing the tool
from autogen import ConversableAgent

agent_retriever = ConversableAgent(
    name="AgentB_MaterialsRetriever",
    system_message=(
        "You are Agent B (Retriever). "
        "Using the user's query and Agent A's explanation, "
        "you must call the tool 'materials_project_query' to query the Materials Project API. "
        "Store the query in context_variables['mp_query'] "
        "and the retrieved results in context_variables['mp_results']."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
)


# -------------------------------------------------------------
# Group Chat Configuration
# -------------------------------------------------------------

In [5]:

from autogen.agentchat.groupchat import GroupChat, GroupChatManager

# GroupChat 
group_chat = GroupChat(
    agents=[agent_explainer, agent_retriever, agent_analyzer],
    speaker_selection_method="round_robin" ,
    max_round=3                              
)

# GroupChatManager 
manager = GroupChatManager(groupchat=group_chat, llm_config=llm_config)

# Store the user query in context variables 
user_query = input("Enter your material design challenge: ")


context_variables["user_query"] = user_query

# Start the conversation FROM Agent A, sending the message to the manager

agent_explainer.initiate_chat(
    manager,
    message=user_query,
    context_variables=context_variables
)



Enter your material design challenge:  Give me a material with high thermal conductivity and low density


AgentA_Explainer (to chat_manager):

Give me a material with high thermal conductivity and low density

--------------------------------------------------------------------------------

Next speaker: AgentB_MaterialsRetriever

AgentB_MaterialsRetriever (to chat_manager):

To find a material with high thermal conductivity and low density, we can query the Materials Project database for materials that have these properties. High thermal conductivity is often desired in applications requiring efficient heat dissipation, while low density is beneficial for lightweight applications.

Let's define the query parameters:
- High thermal conductivity: We can set a minimum threshold, for example, greater than 100 W/mK.
- Low density: We can set a maximum threshold, for example, less than 5 g/cm³.

I'll perform the query to find materials that meet these criteria.

--------------------------------------------------------------------------------

Next speaker: AgentC_Analyzer

AgentC_Analyzer (to c

ChatResult(chat_id=34491863457445640635370706002703372202, chat_history=[{'content': 'Give me a material with high thermal conductivity and low density', 'role': 'assistant', 'name': 'AgentA_Explainer'}, {'content': "To find a material with high thermal conductivity and low density, we can query the Materials Project database for materials that have these properties. High thermal conductivity is often desired in applications requiring efficient heat dissipation, while low density is beneficial for lightweight applications.\n\nLet's define the query parameters:\n- High thermal conductivity: We can set a minimum threshold, for example, greater than 100 W/mK.\n- Low density: We can set a maximum threshold, for example, less than 5 g/cm³.\n\nI'll perform the query to find materials that meet these criteria.", 'name': 'AgentB_MaterialsRetriever', 'role': 'user'}, {'content': 'Please provide the results from the Materials Project database so I can analyze them and determine which materials m

In [ ]:
Give me a material with high thermal conductivity and low density
